In [9]:
# form the spider json 

import json
import pandas as pd

tables = pd.read_json("/blue/daisyw/somasundaramv/DAIL-SQL/dataset/mimic/tables.json")
ops = {0:'=', 1:'>', 2:'<', 3:'>=', 4:'<='}

def get_schema_fk(cur):
    # foreign keys
    fk = []
    if len(cur['table']) == 1:
        fk = None
    else:
        for i in range(0, len(cur['table'])):
            for j in range(i+1, len(cur['table'])):
                fk.append((tables["table_names"][0][i]+".HADM_ID", tables["table_names"][0][j]+".HADM_ID"))    
    
    # conditions
    conditions = []
    if len(cur['cond']) == 0:
        conditions = None
    else:
        offset = {0:0, 1:23, 2:28, 3:33, 4:41}
        for i in range(0, len(cur['cond'])):
            table = tables["table_names"][0][cur['cond'][i][0]]
            column_name = tables["column_names"][0][cur['cond'][i][1] + offset[cur['cond'][i][0]]][1]
            cond = ops[cur['cond'][i][2]]
            value = cur['cond'][i][3]
            conditions.append(table + "." + column_name + " " + cond + " " + str(value))
    return fk, conditions

def convert_to_spider(entry: dict):
    out = {}
    out['db_id'] = "mimic_iv"
    out['query'] = entry['sql']
    out['question'] = entry['question_refine']
    out['question_toks'] = entry['question_refine_tok']
    out["query_toks"] = entry["sql_tok"]
    fk, conditions = get_schema_fk(entry['format'])
    out["fk"] = str(fk)
    out["cond"] = str(conditions)
    return out

def convert(file: str):
    data = [
        # Your input data here
    ]
    
    sql = [
        # sql data here
    ]

    with open(file, 'r') as f:
        line = f.readline()
        while line:
            data.append(json.loads(line))
            line = f.readline()
            
    out = []
    for entry in data:
        cur = convert_to_spider(entry)
        sql.append(cur['query'])
        out.append(cur)

    json.dump(out, open(file.split(".json")[0]+"formatted.json", 'w'), indent=4)
    
    # write the sql to a sql file
    with open(file.split(".json")[0]+".sql", 'w') as f:
        for entry in sql:
            f.write(entry + "\n")

convert("test.json")
convert("train.json")
convert("dev.json")
        
    
